In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**MEMUAT DATA**

In [ ]:
import pandas as pd
import numpy as np

db_berita = pd.read_csv("../input/datasetberita/dataset.csv",delimiter=';', skiprows=0, low_memory=False)

db_berita.head()

Case Folding

In [ ]:
# Case Folding
# mengunakan fungsi Series.srt.lower() yang terdapat pada pandas

db_berita['Artikel']= db_berita['Artikel'].str.lower()


print('Case Folding Result : \n')
print(db_berita['Artikel'].head())
print('\n,\n,\n')

**Tokenizing** (proses numbering removal,whitecase removal,puntuationremoval dan word_tokenizing().)

In [ ]:
import string 
import re #regex library

# import word tokenizing & Fredist dari NLTK
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

#  start tokening


def removal_berita_special(text):
#     hilangkan tab,new lauine , and back slice 
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
#     hilangkan Non ASCII(Emoticon,dll)
    text = text.encode('ascii','replace').decode('ascii')
#     hilangkan mention, link dan hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\s+)"," ",text).split())\
#     menghilangkan url yang tidak lengkap
    return text.replace("http://"," ").replace("https://", " ")

db_berita['Artikel'] = db_berita['Artikel'].apply(removal_berita_special)

# menghilangkan angka
def remove_number(text) :
    return re.sub(r"\d+", "", text)

db_berita['Artikel'] = db_berita['Artikel'].apply(remove_number)

# remove tanda baca
def remove_punctuation(text) :
    return text.translate(str.maketrans("","",string.punctuation))

db_berita['Artikel'] = db_berita['Artikel'].apply(remove_punctuation)

# menghilangkan spasi
def remove_whitespace(text) :
    return re.sub('\s+',' ',text)

db_berita['Artikel'] = db_berita['Artikel'].apply(remove_whitespace)

# menghilangkan karakter tunggal
def remove_singlechar(text) :
    return re.sub(r"\b[a-zA-Z]\b", "",text)

db_berita['Artikel'] = db_berita['Artikel'].apply(remove_singlechar)

# NLTK word Tokenize
def word_tokenize_wrapper(text) :
    return word_tokenize(text)

db_berita['Artikel_tokens'] = db_berita['Artikel'].apply(word_tokenize_wrapper)

print('Tokenizing Result : \n ')
print(db_berita['Artikel_tokens'].head())
print('\n\n\n')

Menghitung frekuensi distribusi token 

In [ ]:
def freqDist_wrapper(text) :
    return FreqDist(text)

db_berita['Artikel_tokens_fdist'] = db_berita['Artikel_tokens'].apply(freqDist_wrapper)

print('Frequency Tokens : \n')
print(db_berita['Artikel_tokens_fdist'].head().apply(lambda x : x.most_common()))

**Filtering(Stopword removal)**

In [ ]:
from nltk.corpus import stopwords

# ambil stopwrod indonesia
list_stopwords = stopwords.words('indonesian')

# menambahkan stopword manual
list_stopwords.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo', 
                       'kalo', 'amp', 'biar', 'bikin', 'bilang', 
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih', 
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't', 
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                       '&amp', 'yah'])

# konvert ke kamus 
def stopwords_removal(word) :
    return [word for word in word if word not in list_stopwords]

db_berita['Artikel_tokens_WSM'] = db_berita['Artikel_tokens'].apply(stopwords_removal)

print(db_berita['Artikel_tokens_WSM'].head())

**Menyimpan Preprocessing Data**

In [ ]:
db_berita.to_csv("db_berita_preprocessing.csv")